In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
!ln -s '/content/drive/My Drive/' /currdrive

In [ ]:
path = '/currdrive/yolov4'
os.chdir(path)

In [ ]:
os.chdir('/currdrive/yolov4/darknet')


In [ ]:
import cv2
import numpy as np

# Load YOLOv4 model
net = cv2.dnn.readNetFromDarknet('path/to/yolov4.cfg', 'path/to/yolov4.weights')

# Get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Load video
video = cv2.VideoCapture('path/to/video.mp4')

# Define classes
classes = ['class1', 'class2', 'class3', ...]  # Replace with actual class names

while True:
    # Read a frame from the video
    ret, frame = video.read()

    if not ret:
        break

    # Resize frame to match YOLO input size
    height, width, _ = frame.shape
    scale = 0.00392
    blob = cv2.dnn.blobFromImage(frame, scale, (416, 416), (0, 0, 0), True, crop=False)

    # Pass the blob through the network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize empty lists for bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []

    # Process each output layer
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Adjust confidence threshold as needed
                # Calculate bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Add bounding box, confidence, and class ID to respective lists
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to eliminate overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)  # Adjust NMS parameters as needed

    # Draw bounding boxes and labels on the frame
    for i in indices:
        i = i[0]
        x, y, w, h = boxes[i]
        label = classes[class_ids[i]]
        confidence = confidences[i]

        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Write label and confidence
        cv2.putText(frame, f'{label}: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Object Tracking', frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()